# Communication, RS-232, ${\rm\bf i^2c}$ and all that. 

## (Controlling an interface remotely with software).

This project has *two* primary goals:

1. Use an external peripheral that communicates with the microcontroller using a serial protocol of some kind (e.g., $i^2c$, SPI, etc.), 
2. to use the microcontroller to collect some data with the notion of making a statistical case that the data collected was legitimate.

You need to select at least *one* $i^2c$, SPI, or serial (RS232) *peripheral* for your project. You also need to make some kind of measurement as a part of your project. 

1. Use an $i^2c$ sensor to measure something. Set up an experiment where you deliberately modify the environment to get a recognizable change in the measured quantity. Use the data collected, and details of the experiment you conducted to argue that your sensor was operating correctly.
2. Use an analog sensor to measure something (e.g., temperature). Use an $i^2c$ display to present the results of those measurements in real time.
3. Use an $i^2c$ joystick to interact with the experiment, send the data collected to the computer over the serial port.

The three most popular approaches to serial communication are RS-232, $i^2c$, and SPI. Let's look at those one at a time.

## RS-232

RS-232 is a three wire "synchronous" serial communication standard designed to provide easy transfer of data between two devices such as a computer and a "dumb terminal (keyboard/screen)" or between two computers. Each endpoint has a minimum of three wires: Transmit (Tx), Receive (Rx) and Ground (Gnd). The two devices need to agree on a predetermined 'data rate' that is generally one of a fixed set of options (measured in 'bits per second'). There are some additional "handshaking" wires that can be used when one device needs extra time bewteen bytes for processing so that it can indicate that it's not prepared for new communication (e.g., Data Set Ready (DTR), Data Terminal Ready (DTR) Ready To Send (RTS), etc.). The good news is that most devices can work well at useful speeds with only three wires, so that's what we'll be doing. If you've used the OpenLog module (think ENGR 196, if you've taken it) you've used an RS-232 connection since that's how that device works. Also, the USB cable we use to program the Arduino becomes a virtual RS-232 connection when you connect using the "Serial Monitor" of the Arduino IDE.

Talking RS-232 from the Arduino
-------------------------------

First consider the following Arduino program:

    /* Sample Program Illustrating Computer Interaction */

    #define LED 13          // pin 13 is the LED
    #define IN_PORT 0       // reading from analog in port 0
    #define NUMLOOPS 5      // every time we get a "g" command we take NUMLOOPS data points
    #define LOOP_DELAY 300  // How long to wait between measurements.

    void setup()
    {
        // start serial port at 9600 bps:
        Serial.begin(9600);
        Serial.println("Hello");  // tell the computer we're ready
        pinMode(LED, OUTPUT);     // set LED pin to output
    }

    void loop() {
      char command;
      int value;

      if (Serial.available()>0) {          // are there characters to read?
        command = (char)Serial.read();     // grab one
        if (command=='g') {                // check it
          Serial.println("Begin");         // it's a "g", let's go!
          for (int i=0; i<NUMLOOPS; i++) {
            value=analogRead(IN_PORT);     // read a data point
            delay(LOOP_DELAY);             // wait for some milliseconds
            Serial.print(i,DEC);           // print the count
            Serial.print(",");             // csv
            Serial.println(value, DEC);    // now the value, and a \cr
          }
          Serial.println("End");           // terminator
        }
      }
      else {
        digitalWrite(LED, HIGH);           // flash our light to let folks know we're waiting.
        delay(LOOP_DELAY);
        digitalWrite(LED, LOW);
        delay(LOOP_DELAY);
      }
    }
    
Let's look at this piece by piece:

    void setup()
    {
        // start serial port at 9600 bps:
        Serial.begin(9600);
        Serial.println("Hello");  // tell the computer we're ready
        pinMode(LED, OUTPUT);     // set LED pin to output
    }

As soon as the Arduino is reset it sets up the serial port to run at 9600 bits per second (this is a pretty standar bit rate for a serial port). Next, we send the signal "Hello" to let the computer know we're ready for commands. Also, set the LED pin as an output so we can let the rest of the world know we're waiting for input.

    void loop() {
      char command;
      int value;

      if (Serial.available()>0) {          // are there characters to read?
        command = (char)Serial.read();     // grab one
        if (command=='g') {                // check it
          Serial.println("Begin");         // it's a "g", let's go!
          for (int i=0; i<NUMLOOPS; i++) {
            value=analogRead(IN_PORT);     // read a data point
            delay(LOOP_DELAY);             // wait for some milliseconds
            Serial.print(i,DEC);           // print the count
            Serial.print(",");             // csv
            Serial.println(value, DEC);    // now the value, and a \cr
          }
          Serial.println("End");           // terminator
        }
      }

Here we first check to see if there are characters available in the serial port's buffer. Serial.available() returns the number of caracters in the buffer. Serial.read() grabs a single character and returns it to the program. We're going to check for a particular character ('g' in this case) and if we see that, we'll do something. In this case we first send "Begin", then enter into a loop that reads voltages on analog pin 0, sends them to the computer, and then finally we send "End".

It's easy to check this program in the "Serial Monitor" of the Arduino program editor. Try it!

Python on the Computer
----------------------

**NOTE**: I used to have students write a program on the computer to talk to the microcontroller over the serial port with software. This is **NO LONGER** a requirement of this project. It's an option if you're interested. You are required to write the microcontroller code so that *it* *could* be controlled by the computer, but it is sufficent to demonstrate that with the [Arduino Serial Monitor](https://learn.adafruit.com/adafruit-arduino-lesson-5-the-serial-monitor/the-serial-monitor), or [Serial Plotter](https://learn.adafruit.com/experimenters-guide-for-metro/circ08-using%20the%20arduino%20serial%20plotter).

Next is an *example of a program for computer to read the data and store it. One of the easiest languages for such work is python: <http://www.python.org>. If you don't already have python on your computer, and you want to experiment with this, you should install it. I would recommend one of the pre-built distributions like anaconda: <https://store.continuum.io/cshop/anaconda/>. For the purposes of this project you'll need to install a serial port module like pyserial. You can install pyserial by using the "pip" command:

    pip install pyserial
    
Consider the following python program (see the comments preceeded by '#' characters):


In [1]:
#
# talk to Arduino! Note, this code needs to be connected to an actual Arduino, locally, to work.
# the Arduino needs to have the program shown above to communicate over the serial port.
# This program also needs to be edited to match the actual serial port of the Arduino!
#

import serial

#port = serial.Serial('COM3')                   # open the serial port (WIN)
port = serial.Serial('/dev/cu.usbserial-1450')  # open the serial port (MAC/Linux), see "PORT" from IDE

print("Waiting for conversation...")

s = port.readline().strip()             # read a line of input from the Arduino

print ("got ",str(s))                   # echo to screen

while s != b'Hello':                    # check for "Hello"
    s = port.readline().strip()         # not yet, try again
    print ("got ",str(s))

port.write(b'g')                        # send a command

s = port.readline().strip()             # get a reply

indexes = []
values = []

while s[:3] != b'End':                  # is it the end?
    print(str(s))                       # nope, echo, repeat
    data = s.decode()
    if ',' in data:
        index,value = data.split(',')
        indexes.append(index)
        values.append(value)
    s = port.readline().strip()
    
print ("Finally got:", str(s))          # that's it!
print ("Finished!")

port.close()                            # shut it down

print(indexes)
print(values)

Waiting for conversation...
got  b'Hello'
b'Begin'
b'0,857'
b'1,857'
b'2,857'
b'3,856'
b'4,855'
Finally got: b'End'
Finished!
['0', '1', '2', '3', '4']
['857', '857', '857', '856', '855']


In [2]:
#
# write values to file
#

f=open('data.csv','w')
f.write("i,val\n")
for i,v in zip(indexes,values):
    f.write("%d,%d\n" % (int(i),int(v))) # old c-style formatting
    
f.close()

 You can see that some of these strings begin with a 'b'. This tells python they should
 be treated as "bytestrings" (essentially arrays of bytes) rather than unicode strings

 Unicode is a [character encoding system](https://docs.python.org/3/howto/unicode.html) that
 allows for a much wider range of characters than simple ASCII, but it requires extra hoops for
 folks like us who just want to send a few bytes over a serial port! In our case the simplest
 way to jump through the hoops is to add a 'b' at the beginning of our strings so notify python
 that we want byte arrays, not unicode strings.
 

In [3]:
#
# Now check that the values are written OK
#
import pandas as pd

df = pd.read_csv('data.csv')
df

,i,val
0,0,857
1,1,857
2,2,857
3,3,856
4,4,855


# ${\rm\bf i^2c}$

In order to permit a large number of devices with a small number of "pins" $i^2c$ was developed [in 1982](https://en.wikipedia.org/wiki/I²C). Basically the "serial bus" system uses two wires (SCL [clock], SDA [data]) which are never driven "high", but only ever pulled "low" by systems sharing the "bus". The advantage of this is that there is never any risk of two devices "fighting" over control in a way that could result in shorts or power being delivered from one device to another. Each wire is "pulled high" by a resistor (usually $5 k\Omega$) or so).

The easiest way to access $i^2c$ peripherals on the Arduino is to use the ["Wire"](https://www.arduino.cc/en/reference/wire) library (see the example link below). 

[i2c tutorial](https://learn.sparkfun.com/tutorials/i2c "An $i^2c$ Tutorial")

[example](https://howtomechatronics.com/tutorials/arduino/how-i2c-communication-works-and-how-to-use-it-with-arduino/)

# The SPI Interface

Yet another approach that's often used is the Serial Peripheral Interface (SPI). This is not as expandable/flexible as $i^2c$, but it's fast and popular. You can read about the [Arduino library for SPI](https://www.arduino.cc/en/Reference/SPI) in the [reference](https://www.arduino.cc/reference/en/) section of the [Arduino](https://www.arduino.cc) website.

Deliverables
------------

Your objective is to control the Arduino over the serial port to measure some kind of interesting value over a period of one or two minutes (maybe sampling once every  second or two? You should have *tens* of data points, or maybe a hundred or so at the *most*. No need for thousands of data values.) Do something to make your data vary a little during the "experiment". Please use at least one $i^2c$, SPI, or RS232 based peripheral in this experiment. One option is to measure temperature using an $i^2c$ temperature probe, or some other $i^2c$ based breakout board (e.g., altitude, humidity, light level, environmental sensor), or you could use a probe that produces a voltage, but store the data in an $i^2c$ or SPI based memory. You could also create a program that controlled something (e.g., a motor, switch, etc.) using $i^2c$. Be creative! Surprise me.

You should produce a standard report. Be sure to explain what you measured and how you determined that it was "working". There's more about that in the [Statistics Exercises](The%20Mystery%20of%20Statistical%20Significance.ipynb) for this week.